## TOOL UPDATED: December 12 2023, Henrik Loecke

##Double click this cell to see full description.

##You must restart the kernel after updating System_Assessment_Variables!

<!-- 

To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open System_Assessment_Generate_Models_Variables, in the same folder as this notebook, to edit user input there.

All variables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'  
generate_future:                     Generate future population models (True/False)
                                     If True, the same models will be used for the other models (BSF etc.)
                                     If False, the other models will instead be made from all models in the folder.
generate_bsf:                        Generate BSF models (True/False)
generate_xadwf:                      Generate X times ADWF models (True/False)
generate_sealed_vfd                  Generate sealed and/or VFD models (True/False)
generate_AD                          Generate AD models (True/False)

output_folder:                       Where the new models are created. It is recommended to use an empty folder.
script_path:                         Path to a template of the WaterspillDischarge.cs script.

#For all models:
version:                             Model version from the backup log.

#For future models
model_original:                      The model copied for future population models. It is usually the master model.
year_original:                       The original population year. It is used for search/replace to the new year.
year_scenario_list:                  List of lists of years to create, which scenario to use and which to include to batch.
population_sheet:                    The source population workbook.
population_tab:                      The sheet name within the source population sheet.

#For sealed VFD
vfd_all:                             Create VFD for all pump stations (except the exclusions below)
seal_all:                            Seal all nodes and deactivate weirs listed below.
excluded_asset_names:                List of PS asset name
weir_turn_offs:                      Weirs to deactivate if VFD model is made.

#For BSF
inis                                 List of inis in m3/ha,d. Example for 1 and 2 I/I: [11.2,22.4]

#For X ADWF
times_adwf_list                      List of X ADWF. Example for 3 and 4 ADWF: [3,4]

#For AD
dfs0_file:                           Dfs0 file with runoff result (used if runoff is traced).
model_suffix:                        Suffix added to the model name.
tracer_csv:                          Csv file with tracer zones.
boundary_tracer_ext:                 Suffix to tracer boundary conditions (Should be short to keep MUID length below 41).
sim_ext:                             Suffix added to the simulation ID.
runoff_tracer                        Toggle runoff tracer.
ww_tracer                            Toggle wastewater tracer.
gwi_tracer                           Toggle gwi tracer.
include_gwi_in_ww                    Toggle whether gwi included in wastewater tracer.
tracer_from_original_zone            Toggle whether the tracer is from the original zone.
global_zone_only                     Toggle whether to use global tracer only.
single_tracer                        Toggle whether to use single tracer only.
single_tracer_ids                    List the single tracer ids (suggest ['All','All'])
file_suffix = "_AD"                  File name suffix.

 -->

In [ ]:
#PERMANENT CELL 1

import sqlite3
import numpy as np
import pandas as pd
import ctypes
import shutil
import os
MessageBox = ctypes.windll.user32.MessageBoxW
from System_Assessment_Generate_Models_Variables import *
import traceback



In [ ]:
#Permanent cell 2
#Functions

def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            print(sql)
            cur.execute(sql)
    else:
        sql = sqls
        print(sql)
        cur.execute(sql)
    cur.close()
    con.commit()
    con.close()
  
def df_to_sql(df,table_name,model):
    conn = sqlite3.connect(model)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    
def generate_script(script_path,mu_path):  
    script_path_new = os.path.splitext(mu_path)[0] + '.cs'
    shutil.copy(script_path, script_path_new)
    
def generate_mupp(model_original,mu_path):
    mupp_path_original = os.path.splitext(model_original)[0] + '.mupp'
    mupp_path_new = os.path.splitext(mu_path)[0] + '.mupp'   
    with open(mupp_path_original, 'r') as source_file, open(mupp_path_new, 'w') as output_file:
        for line in source_file:
            if line.startswith(r'   DBFilePath = |.'):
                output_file.write('   DBFilePath = |.\\' + os.path.splitext(os.path.basename(mu_path))[0] + '.sqlite|\n')
            else:
                output_file.write(line)
                
    

In [ ]:
#Permanent cell 3
#Early error catching and initialization

mu_paths_append = []

if generate_bsf:
    for ini in inis:
        if np.mod(ini,11.2) > 0:
            message = 'ini ' + str(ini) + ' is not a multiple of 11.2\n\nContinue?'
            if MessageBox(None, message, 'Warning', 4) == 7:
                MessageBox(None, "Please correct inis", 'Info', 0)
                raise ValueError(message)
            else:
                pass
            


    

In [ ]:
#Permanent cell 4
#Generate future models
try:
    if generate_future:

        mu_paths = []

        keep_cols = ['Catchment','Zone','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Area_Com','Area_Ind','Area_Inst']
        pop_df = pd.read_excel(population_sheet,sheet_name=population_tab,usecols=keep_cols,dtype={'Catchment': str})

        # Melt the DataFrame to combine the columns
        columns_to_combine = ['Pop_ResLD', 'Pop_ResHD', 'Pop_Mixed', 'Area_Com', 'Area_Ind', 'Area_Inst']
        pop_df_melt = pd.melt(pop_df, id_vars=['Catchment', 'Zone', 'Year'], value_vars=columns_to_combine, var_name='Type', value_name='Value')
        pop_df_melt[['Major_type', 'Minor_type']] = pop_df_melt['Type'].str.split('_', n=1, expand=True)
        pop_df_melt = pop_df_melt.drop(columns=['Type'])
        pop_df_melt['MUID'] = pop_df_melt.Catchment + '_' + pop_df_melt.Minor_type
        pop_df_melt

        #Check for missing catchments
        catchment_years = []
        sql = "SELECT catchmentid FROM msm_Loadpoint GROUP BY catchmentid"
        muids = list(sql_to_df(sql,model_original).catchmentid)

        for muid in muids:
            for year_scenario in year_scenario_list:
                year = year_scenario[0]
                catchment_years.append([muid,year])
        catchment_year_df = pd.DataFrame(catchment_years,columns=(['Catchment','Year']))
        catchment_year_df 

        merged = catchment_year_df.merge(pop_df[['Catchment', 'Year']], on=['Catchment', 'Year'], how='left', indicator=True)

        not_founds = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

        if len(not_founds) > 0:
            message = "WARNING.The following catchment/year combinations are not found\n\n"
            for index, row in not_founds.iterrows():
                message += row[0] + ', ' + str(row[1]) + '.\n'
            message += '\nContinue?'

            if MessageBox(None, message, 'Warning', 4) == 7:
                MessageBox(None, "Please report the missing catchment(s)", 'Info', 0)
                raise ValueError(message)
            else:
                pass

        for year_scenario in year_scenario_list:
            year = year_scenario[0]
            scenario = year_scenario[1]
            turnons = year_scenario[2]

            model_name = model_area + '_' + str(year) + 'pop_V' + str(version) + '.sqlite'

            if os.path.basename(model_original) == model_name:

                message = "Tool ends. For year " + str(year) + ", the new model name '" + model_name + "' is the same as the original."
                MessageBox(None, message, 'Info', 0)
                raise ValueError("message")

            #Delete sqlite, mupp and cs if they exist and create new
            mu_path = output_folder + "\\" + model_name

            mu_paths.append(mu_path)

            generate_script(script_path,mu_path)
            generate_mupp(model_original,mu_path)

            os.remove(mu_path) if os.path.exists(mu_path) else None
            shutil.copyfile(model_original, mu_path)

            if year != year_original:

                sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
                muids = list(sql_to_df(sql,mu_path).muid)
                for muid in muids:
                    muid_new = muid.replace(str(year_original) + 'p',str(year) + 'p')
                    sql = "UPDATE msm_Project SET muid = '" + muid_new + "' WHERE muid = '" + muid + "'"
                    execute_sql(sql, mu_path)

                    sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"
                    execute_sql(sql, mu_path)

                sql = "UPDATE msm_Project SET IncludeToBatchNo = 0 WHERE MUID LIKE '%h-AES_%'"
                execute_sql(sql, mu_path)

                sql = "UPDATE msm_Project SET scenarioname = '" + scenario + "' WHERE MUID NOT LIKE '%h-AES_%' AND enable_hd = 1"
                execute_sql(sql, mu_path)

                for turnon in turnons:
                    sql = "UPDATE msm_Project SET IncludeToBatchNo = 1 WHERE MUID LIKE '%h-AES_%' AND MUID LIKE '%" + turnon + "%' "
                    sql += "AND SUBSTR(scenarioname,1," + str(len(scenario)) + ") = '" + scenario  + "'"
                    execute_sql(sql, mu_path)

                pop_df_melt_year = pop_df_melt[pop_df_melt.Year==year]

                df_to_sql(pop_df_melt_year,'New_Population',mu_path)

                sql = "UPDATE msm_Loadpoint SET Population = "
                sql += "(SELECT Value FROM New_Population WHERE MUID = msm_Loadpoint.muid AND Major_Type = 'Pop')"
                execute_sql(sql, mu_path)

                sql = "UPDATE msm_Loadpoint SET ICIArea = "
                sql += "(SELECT Value FROM New_Population WHERE MUID = msm_Loadpoint.muid AND Major_Type = 'Area')"
                execute_sql(sql, mu_path)

                sql = "UPDATE msm_Loadpoint SET loadflow = PerCapitaLoad * Population / 86400 WHERE LoadCategory = 'Mixed' OR LoadCategory = 'ResLD' OR LoadCategory = 'ResHD'"
                execute_sql(sql, mu_path)
                sql = "UPDATE msm_Loadpoint SET loadflow = PerAreaLoad * ICIArea / 86400 WHERE LoadCategory = 'Commercial' OR LoadCategory = 'Industrial' OR LoadCategory = 'Institutional'"
                execute_sql(sql, mu_path)

                sql = "DROP TABLE New_Population"
                execute_sql(sql, mu_path)

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,'An error happened in permanent cell 4', 'Error', 0)
    raise ValueError("Error")


In [ ]:
#Permanent cell 5
#Generate BSF models
try:
    if generate_bsf:

        if not generate_future:
            mu_paths = []
            for f in os.listdir(output_folder):
                if f[-7:]==".sqlite" and not 'BSF' in f and not 'xADWF' in f:
                    mu_paths.append(output_folder + '\\' + f)


        for mu_path_original in mu_paths:

            sqls = []

            pop_pos = mu_path_original.lower().find('pop')
            year = mu_path_original[pop_pos-4:pop_pos]

            for ini in inis:

                ini_str = str(ini).replace('.','p')

                mu_path = mu_path_original[:-7]  + "_BSF_" + ini_str + "k.sqlite"

                mu_paths_append.append(mu_path)

                os.remove(mu_path) if os.path.exists(mu_path) else None
                shutil.copyfile(mu_path_original, mu_path)

                print('Generating ' + mu_path)

                generate_script(script_path,mu_path)
                generate_mupp(mu_path_original,mu_path)

                sql = "SELECT muid, area, nettypeno FROM msm_Catchment WHERE nettypeno <> 2"
                catchments = sql_to_df(sql,mu_path)

                sqls = []
                for index, row in catchments.iterrows():
                    MUID = str(row[0])
                    Area = row[1]
                    NetTypeNo = str(row[2])

                    flow = Area * ini

                    sql = "UPDATE msm_LoadPoint SET MUID = '" + MUID + "_BSF', loadflow = " + str(flow / 86400 / 10000) + ", Description = 'BSF', LoadCategoryNo = 1, LoadCategory = 'BSF', LoadSubCategory = 'BSF_' & LoadLocation "
                    sql += "WHERE MUID = '" + MUID + "_Load_8'"

                    sqls.append(sql)

                execute_sql(sqls, mu_path)

                muid_new = model_area + "_BSF_" + ini_str + "k_" + year + "pop_"

                sql =  "DELETE FROM msm_Project WHERE enable_catchment = 1"
                execute_sql(sql, mu_path)

                sql = "SELECT muid FROM msm_Project"
                sims = sql_to_df(sql,mu_path)

                for index, row in sims.iterrows():
                    muid = row[0]
                    if index == 0:
                        sql = "UPDATE msm_Project SET MUID = '" + muid_new + "', Description = 'BSF' WHERE MUID = '" + muid + "'"
                        execute_sql(sql, mu_path)
                        sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"          
                        execute_sql(sql, mu_path)
                    else:
                        sql =  "DELETE FROM msm_Project WHERE simulationid = '" + muid + "'"
                        execute_sql(sql, mu_path)
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,'An error happened in permanent cell 5', b'Error', 0)
    raise ValueError("Error")


In [ ]:
#Permanent cell 6
#Generate X ADWF models
try:
    if generate_xadwf:
        deficit_list = []
        diurnals = []

        if not generate_future:
            mu_paths = []
            for f in os.listdir(output_folder):
                if f[-7:]==".sqlite" and not 'BSF' in f and not 'xADWF' in f:
                    mu_paths.append(output_folder + '\\' + f)

        for i, mu_path_original in enumerate(mu_paths):

            sqls = []

            pop_pos = mu_path_original.lower().find('pop')
            year = mu_path_original[pop_pos-4:pop_pos]

            sql = "SELECT Sum(loadflow) AS WaterLoad FROM msm_loadpoint WHERE loadcategory = 'Baseflow'"
            gwi_global = sql_to_df(sql,mu_path_original).iloc[0,0]
    #         gwi_global = list(sql_to_df(sql,mu_path_original).WaterLoad)[0]


            sql = "SELECT ms_DPProfileD.ScheduleID AS Day_Type, ms_DPPatternD.Sqn AS [Hour], Sum(msm_Loadpoint.loadflow*ms_DPPatternD.DPValue) + " + str(gwi_global) + " AS Discharge "
            sql += "FROM ((msm_Loadpoint INNER JOIN msm_BBoundary ON msm_Loadpoint.LoadCategoryNo = msm_BBoundary.LoadCategoryNo) INNER JOIN ms_DPProfileD ON msm_BBoundary.DPProfileID = ms_DPProfileD.ProfileID) INNER JOIN ms_DPPatternD ON ms_DPProfileD.PatternID = ms_DPPatternD.PatternID "
            sql += "WHERE msm_Loadpoint.Active = 1 AND ms_DPProfileD.Active = 1 AND ms_DPPatternD.Active = 1 AND msm_BBoundary.Active = 1 "
            sql += "GROUP BY ms_DPProfileD.ScheduleID, ms_DPPatternD.Time "
            sql += "HAVING (LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekday' Or LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekend') AND ms_DPPatternD.Sqn <> 0 "
            sql += "ORDER BY scheduleid, time"

            diurnal_wws = sql_to_df(sql,mu_path_original)        
            diurnal_wws.Hour = diurnal_wws.index
            diurnal_wws.loc[diurnal_wws.index > 23, 'HOUR'] = diurnal_wws.index[diurnal_wws.index > 23] - 24

            for times_adwf in times_adwf_list:

                mu_path = mu_path_original[:-7]  + "_" + str(times_adwf).replace('.','p') + "xADWF.sqlite"

                mu_paths_append.append(mu_path)

                os.remove(mu_path) if os.path.exists(mu_path) else None
                shutil.copyfile(mu_path_original, mu_path)

                print('Generating ' + mu_path)

                generate_script(script_path,mu_path)
                generate_mupp(mu_path_original,mu_path)

                sqls = []

                sql = "SELECT muid FROM msm_Catchment"
                muids = list(sql_to_df(sql,mu_path).muid)
                for muid in muids:

                    sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE CatchmentID = '" + muid + "' AND msm_Loadpoint.Active = 1"
                    adwf = sql_to_df(sql,mu_path).iloc[0,0]

                    sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE CatchmentID = '" + muid + "' AND LoadCategory = 'Baseflow' AND msm_Loadpoint.Active = 1"
                    gwi = sql_to_df(sql,mu_path).iloc[0,0]

                    sql = "SELECT Max(SumOfDPValue) "
                    sql += "FROM (SELECT "
                    sql += "ms_DPProfileD.ProfileID, Sum(msm_Loadpoint.loadflow*ms_DPPatternD.DPValue) AS SumOfDPValue "
                    sql += "FROM ((msm_Loadpoint INNER JOIN msm_BBoundary ON msm_Loadpoint.LoadCategoryNo = msm_BBoundary.LoadCategoryNo) INNER JOIN ms_DPProfileD ON msm_BBoundary.DPProfileID = ms_DPProfileD.ProfileID) INNER JOIN ms_DPPatternD ON ms_DPProfileD.PatternID = ms_DPPatternD.PatternID "
                    sql += "WHERE msm_Loadpoint.Active = 1 And msm_BBoundary.Active = 1 And ms_DPProfileD.Active = 1 And ms_DPPatternD.Active = 1 "
                    sql += "GROUP BY ms_DPProfileD.ProfileID,ms_DPPatternD.MUID, msm_Loadpoint.CatchmentID, ms_DPProfileD.ScheduleID "
                    sql += "HAVING msm_Loadpoint.CatchmentID = '" + muid + "' AND (LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekday' Or LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekend'))"

                    pww = sql_to_df(sql,mu_path).iloc[0,0]
                    pdwf = gwi + pww
                    deficit = times_adwf * adwf - pdwf

                    deficit_list.append([os.path.basename(mu_path),muid,deficit])

                    sql = "UPDATE msm_Loadpoint SET muid = '" + muid + "_" + str(times_adwf).replace('.','p') + "xADWF', "
                    sql += "loadflow = " + str(deficit) + ", Description = '" + str(times_adwf).replace('.','p') + "xADWF', "
                    sql += "LoadCategoryNo = 1, LoadCategory = 'X-ADWF', LoadSubCategory = 'X-ADWF_' & LoadLocation "
                    sql += "WHERE muid = '" + muid + "_Load_8'"
                    sqls.append(sql)

                execute_sql(sqls, mu_path)

                muid_new = model_area + "_" + str(times_adwf).replace('.','p') + "ADWF_" + year + "pop_"
                sql =  "DELETE FROM msm_Project WHERE enable_catchment = 1"
                execute_sql(sql, mu_path)
                sql = "SELECT muid FROM msm_Project"
                sims = sql_to_df(sql,mu_path)
                for index, row in sims.iterrows():
                    muid = row[0]
                    if index == 0:
                        sql = "UPDATE msm_Project SET MUID = '" + muid_new + "', Description = 'BSF' WHERE MUID = '" + muid + "'"
                        execute_sql(sql, mu_path)
                        sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"          
                        execute_sql(sql, mu_path)
                    else:
                        sql =  "DELETE FROM msm_Project WHERE simulationid = '" + muid + "'"
                        execute_sql(sql, mu_path)

                sql = "SELECT Sum(loadflow) FROM msm_Loadpoint WHERE LoadCategory = 'X-ADWF' AND Active = 1"
                deficit = sql_to_df(sql,mu_path).iloc[0,0]

                diurnal_wws['Model'] = os.path.basename(mu_path)
                diurnal_wws['Deficit'] = deficit
                diurnal_wws = diurnal_wws[['Model','Day_Type','Hour','Discharge','Deficit']]

                if i == 0:
                    diurnals_df = diurnal_wws.copy()
                else:
                    diurnals_df = pd.concat([diurnals_df,diurnal_wws])           

        deficit_df = pd.DataFrame(deficit_list,columns=['Model','Catchment','Deficit'])
        deficit_df.to_csv(output_folder + '\\Deficits.csv',index=False)

        diurnals_df = pd.DataFrame(diurnals,columns=['Model','Day_Type','Hour','Discharge','Deficit'])
        diurnals_df.to_csv(output_folder + '\\X-ADWF_Diurnals.csv',index=False)

    print('Done')
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,'An error happened in permanent cell 6', b'Error', 0)
    raise ValueError("Error")

In [ ]:
#Permanent cell 7
#Generate sealed vfd models
try:
    if generate_sealed_vfd:

        if not generate_future:
            mu_paths = []
            for f in os.listdir(output_folder):
                if f[-7:]==".sqlite" and not 'VFD' in f:
                    mu_paths.append(output_folder + '\\' + f)
        else:
            mu_paths += mu_paths_append

        if seal_all and vfd_all:
            suffix = "S_V_"
            file_suffix = "_Sealed_VFD"
        elif seal_all and not vfd_all:
            suffix = "S_"
            file_suffix = "_Sealed"
        elif not seal_all and vfd_all:        
            suffix = "V_"
            file_suffix = "_VFD" 
        else:
            message = "Error! generate_sealed_vfd set to True but both vfd_all and seal_all set to False"
            raise ValueError(message)

        for mu_path_original in mu_paths:

            sqls = []

            mu_path = mu_path_original[:-7] + file_suffix + ".sqlite"
            os.remove(mu_path) if os.path.exists(mu_path) else None
            shutil.copyfile(mu_path_original, mu_path)

            generate_script(script_path,mu_path)
            generate_mupp(mu_path_original,mu_path)

            print('Generating ' + mu_path)

            if seal_all:
                sqls.append("UPDATE msm_Node SET covertypeno = 2")
                for weir_turn_off in weir_turn_offs:
                    sqls.append("UPDATE msm_Weir SET crestlevel = crestlevel + 1000 WHERE muid = '" + weir_turn_off + "'")

            if vfd_all:

                pumps_for_VFD = []
                pumps_turn_off = []

                sqls.append("INSERT INTO ms_Tab (muid,altid,active,description,typeno) SELECT 'Generic_Pump_Min',0,1,'Use for PS passing all inflow',2")
                sqls.append("INSERT INTO ms_Tab (muid,altid,active,description,typeno) SELECT 'Generic_Pump_Max',0,1,'Use for PS passing all inflow',2")

                sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Max-1',0,1,'Generic_Pump_Max',1, 0, 50")
                sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Max-2',0,1,'Generic_Pump_Max',2, 100, 50")

                sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Min-1',0,1,'Generic_Pump_Min',1, 0, 0")
                sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Min-2',0,1,'Generic_Pump_Min',2, 100, 0")

                sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
                muids = list(sql_to_df(sql,mu_path).muid)
                for muid in muids:
                    if len(muid) <= 40 - len(suffix):
                        muid_new = muid + suffix
                        sqls.append("UPDATE msm_Project SET muid = '" + muid_new + "' WHERE muid = '" + muid + "'")        
                        sqls.append("UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'")            

                sql = "SELECT assetname, muid, startlevel from msm_Pump ORDER BY assetname, startlevel"
                pumps = sql_to_df(sql,mu_path)

                previous_asset = 'xxxx'
                for index, row in pumps.iterrows():
                    asset = str(row[0])
                    if not asset in excluded_asset_names and asset != 'None':
                        muid = str(row[1])
                        if previous_asset != asset:
                            pumps_for_VFD.append(muid)
                        else:
                            pumps_turn_off.append (muid)
                        previous_asset = asset
                    else:
                        print ('Skipped ' + str(row[1]))

                for pump_for_VFD in pumps_for_VFD:
                    sqls.append("UPDATE msm_Pump SET qmaxsetid = 'Generic_Pump_Max',  qminsetid = 'Generic_Pump_Min', speedno = 2, captypeno = 2, regno = 1, controltypeno = 2, startlevel = stoplevel + 0.1,  wetwellsetpoint = stoplevel + 0.1  WHERE muid = '" + pump_for_VFD + "'")
                    sqls.append("UPDATE msm_RTC SET applyno = 0 WHERE pumpid = '" + pump_for_VFD + "'")

                for pump_turn_off in pumps_turn_off:
                    sqls.append("UPDATE msm_Pump SET startlevel = startlevel + 100, stoplevel = stoplevel + 100,  controltypeno = 1 WHERE MUID = '" + pump_turn_off + "'")
                    sqls.append("UPDATE msm_RTC SET applyno = 0 WHERE pumpid = '" + pump_turn_off + "'")

            execute_sql(sqls, mu_path)
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,'An error happened in permanent cell 7', b'Error', 0)
    raise ValueError("Error")


In [ ]:
#PERMANENT CELL 8
#Generate AD models.


if generate_ad:
    
    if not generate_future:
        mu_paths = []
        for f in os.listdir(output_folder):
            if f[-7:]==".sqlite" and not 'AD' in f:
                mu_paths.append(output_folder + '\\' + f)

    for i, mu_path_original in enumerate(mu_paths):

        mu_path = mu_path_original[:-7] + file_suffix + ".sqlite"
        os.remove(mu_path) if os.path.exists(mu_path) else None
        shutil.copyfile(mu_path_original, mu_path)

        generate_script(script_path,mu_path)
        generate_mupp(mu_path_original,mu_path)

        sql = "UPDATE m_ModelSetting SET enable_ad = 1"
        execute_sql(sql, mu_path)
        
        #Fill tracer zone
        sql = "ALTER TABLE msm_Catchment ADD COLUMN Tracer TEXT"
        execute_sql(sql, mu_path)
        
        sql = "ALTER TABLE msm_Catchment ADD COLUMN TracerShort TEXT"
        execute_sql(sql, mu_path)
        
        
        if single_tracer:
            tracer = single_tracer_ids[0]
            tracer_z = single_tracer_ids[1]
            sql = "UPDATE msm_Catchment SET Tracer = '" + tracer + "', TracerShort = '" + tracer_z + "'"
            execute_sql(sql, mu_path)
        else:
            sqls = []
            for i, row in tracer_ids.iterrows():
                muid = str(row["MUID"])
                tracer = str(row["Tracer"])
                tracer_z = str(row["Tracer_Short"])
                sqls.append("UPDATE msm_Catchment SET Tracer = '" + tracer + "', TracerShort = '" + tracer_z + "'  WHERE MUID = '" + muid + "'")
            execute_sql(sqls, mu_path)
        
        sql = "CREATE TABLE Tracer_Zones AS "
        sql += "SELECT Location, Tracer, Location || '_' || TracerShort AS TracerZone "
        sql += "FROM msm_Catchment WHERE Active = 1 "
        sql += "GROUP BY Location, Tracer, TracerZone"
        execute_sql(sql, mu_path)

        sql = "SELECT * FROM Tracer_Zones"
        tracer_zones = sql_to_df(sql,mu_path)

        sql = "SELECT CatchID, NodeID, Location, tracer FROM msm_Catchcon INNER JOIN msm_Catchment ON msm_Catchment.MUID = msm_Catchcon.CatchID"
        catchments = sql_to_df(sql,mu_path)

        tracers = set()
        zones = set()
        boundary_names_too_long = set()
        old_zones = set()

        #

        if tracer_from_original_zone == False:
            #Create new boundary conditions.

            b_type = 'TypeNo'

            sqls = []


            for index, row in tracer_zones.iterrows():

                zone = row['location']
                old_zones.add(zone)
                

                tracer = row['Tracer']
                tracer_z = row['TracerZone']

                sql = "CREATE TABLE BTemp AS SELECT * FROM msm_BBoundary WHERE TypeNo = 7 "
                sql += "AND (SUBSTR(MUID, -" + str(len(zone)+1) + ") = '_" + zone + "' OR MUID = 'Baseflow') AND Active = 1"
                sqls.append(sql)
                               
                sql = "UPDATE BTemp SET MUID = SUBSTR(MUID, 1, LENGTH(MUID) -  " + str(len(zone)+1) + ") || '_" + tracer_z + "' WHERE MUID <> 'Baseflow'"
                sqls.append(sql)
                

                sql = "UPDATE BTemp SET MUID = 'Baseflow_" + tracer_z + "' WHERE MUID = 'Baseflow'"
                sqls.append(sql)

                if i == 0:
                    sql = "UPDATE BTemp SET Description = '" + tracer + "'"
                    sqls.append(sql)


                sql = "INSERT INTO msm_BBoundary SELECT * FROM BTemp"
                sqls.append(sql)

                sql = "DROP TABLE BTemp"
                sqls.append(sql)

            for old_zone in old_zones:
                sql = "DELETE FROM msm_BBoundary WHERE TypeNo = 7 AND SUBSTR(MUID, -" + str(len(old_zone)) + ") = '" + old_zone + "'"
                sqls.append(sql)

            sql = "DELETE FROM msm_BBoundary WHERE MUID = 'Baseflow'"
            sqls.append(sql)

            execute_sql(sqls, mu_path)


        sqls = []
        for c in catchments:
            catchment = str(c[0])
            node = str(c[1])
            zone = str(c[2])

            if global_zone_only == True:
                tracer = "Runoff"
            else:
                tracer = "Runoff_" + str(c[3])

            zones.add(zone)

            #For later uncomment and update

#             if runoff_tracer == True:

#                 tracers.add(tracer)

#                 boundary_id = 'Runoff_Inflow_' + catchment
#                 boundary_tracer_id = boundary_id + boundary_tracer_ext

#                 if len(boundary_id) > 40:
#                     boundary_names_too_long.add(boundary_id)
#                 if len(boundary_tracer_id) > 40:
#                     boundary_names_too_long.add(boundary_tracer_id)

#                 sql = "INSERT INTO msm_BBoundary "
#                 sql += "(MUID, ApplyBoundaryNo, GroupNo, TypeNo, ConnectionTypeNo, SourceLocationNo, IndividualConnectionNo, NodeID, NodeLoadTypeNo, CatchLoadNo, OpenBoundaryNo, Kmix, DistributeNo, LoadCategoryNo, GridTypeNo) "
#                 sql += "SELECT '" + boundary_id + "', 1, 2, 5, 3, 0, 1, '" + node + "', 1, 0, 0, 0.5, 0, 1, 1 "
#                 sqls.append(sql)

#                 sql = "INSERT INTO msm_BItem "
#                 sql += "(MUID, Description, BoundaryID, BoundaryType, TypeNo, TrapComponentID, Fraction, LoadTypeNo, VariationNo, LoadModelNo, ConstantValue, "
#                 sql += "StartUpNo, StartUpTime, WholeFileNo, ValidityIntervalNo, ValidityBegin, ValidityEnd) "
#                 sql += "SELECT '" + boundary_id + "_T', '" + boundary_tracer_id + "', '" + boundary_id + "', 5, 2, '" + tracer + "', 1, 1, 1, 3, 1, 0, 60, 1, 0, 2, 73051"
#                 sqls.append(sql)

#                 sql = "INSERT INTO msm_BItem "
#                 sql += "(MUID, Description, BoundaryID, BoundaryType, TypeNo, Fraction, LoadTypeNo, VariationNo, StartUpNo, StartUpTime, BridgeTypeNo, TSConnection, DataTypeName, TimeseriesName, "
#                 sql += "WholeFileNo, ValidityIntervalNo, ValidityBegin, ValidityEnd) "
#                 sql += "SELECT '" + boundary_id + "', '" + boundary_id + "', '" + boundary_id + "', 5, 1, 1, 3, 3, 0, 60, 2, '" + dfs0_file + "', 'Discharge', '" + catchment + "', 1, 0, 2, 73051"
#                 sqls.append(sql)


        if ww_tracer == True:

            if include_gwi_in_ww == True:
                sql = "SELECT MUID, SUBSTR(MUID, INSTR(MUID, '_') + 1) AS Zone, Description, LoadTypeNo FROM msm_BBoundary WHERE TypeNo = 7 AND Active = 1"
            else:
                sql = "SELECT MUID, SUBSTR(MUID, INSTR(MUID, '_') + 1) AS Zone, Description, LoadTypeNo FROM msm_BBoundary WHERE SUBSTR(MUID,-8) <> 'Baseflow' AND TypeNo = 7 AND Active = 1"
            ww_boundaries = sql_to_df(sql,mu_path)

            for index, row in ww_boundaries.iterrows():
                boundary_id = row['muid']
                boundary_tracer_id = boundary_id + boundary_tracer_ext
                tracer = row['description']
                load_type = row['loadtypeno']

                if len(boundary_id) > 40:
                    boundary_names_too_long.add(boundary_id)
                if len(boundary_tracer_id) > 40:
                    boundary_names_too_long.add(boundary_tracer_id)

                if global_zone_only == True:
                    zone = "Wastewater"
                    tracer = "Wastewater"
                else:
                    zone = str(ww_boundary[1])
                    tracer = "Wastewater_" + tracer

                if load_type == 3:
                    make_sustom_tracer = False
                    for custom_tracer in custom_tracers:
                        if boundary_id == custom_tracer[1]:
                            make_sustom_tracer = True
                            tracer = custom_tracer[0]
                    if make_sustom_tracer == False:
                        continue

                tracers.add(tracer)

                sql = "INSERT INTO msm_WQBoundaryProperties "
                sql += "(muid,altid,active,enabled,boundaryid,wqboundarytypeno,variationno,constantvalue,cyclicvalue,"
                sql += "trapcomponentid,startupno,startupvalue,startuptime,fraction) "
                sql += "SELECT '" + boundary_tracer_id + "',0,1,1,'" + boundary_id + "',1,1,1,0,'" + tracer + "',0,0,60,1"
                sqls.append(sql)
#                 sql = "UPDATE msm_BItem SET Description = MUID WHERE MUID = '" + boundary_id + "'"
#                 sqls.append(sql)
   
        sql = "SELECT SUBSTR(MUID, INSTR(MUID, '_') + 1) AS Zone FROM msm_BBoundary "
        sql += "WHERE MUID <> 'Baseflow' AND ConnectionTypeNo = 5 GROUP BY SUBSTR(MUID, INSTR(MUID, '_') + 1)"
        zones = list(sql_to_df(sql,mu_path).Zone.unique())


        if gwi_tracer == True and include_gwi_in_ww == False:

            sql = "SELECT MUID, SUBSTR(MUID, INSTR(MUID, '_') + 1) AS Zone, Description FROM msm_BItem "
            sql += "WHERE SUBSTR(MUID, 1, 8) = 'Baseflow' AND TypeNo = 7"
            gwi_boundaries = sql_to_df(sql,mu_path)

            for index, row in gwi_boundaries.iterrows():

                boundary_id =row['MUID']
                boundary_tracer_id = boundary_id + boundary_tracer_ext
                tracer = str(gwi_boundary[2])

                if len(boundary_id) > 40:
                    boundary_names_too_long.add(boundary_id)
                if len(boundary_tracer_id) > 40:
                    boundary_names_too_long.add(boundary_tracer_id)

                if global_zone_only == True:
                    tracer = 'GWI'
                    tracers.add(tracer)

                else:
                    tracer = "GWI_" + tracer

                    #for zone in zones:
                    #tracer = "GWI_" + tracer
                    tracers.add(tracer)

                sql = "INSERT INTO msm_WQBoundaryProperties "
                sql += "(muid,altid,active,enabled,boundaryid,wqboundarytypeno,variationno,constantvalue,cyclicvalue,"
                sql += "trapcomponentid,startupno,startupvalue,startuptime,fraction) "
                sql += "SELECT '" + boundary_tracer_id + "',0,1,1,'" + boundary_id + "',1,1,1,0,'" + tracer + "',0,0,60,1"
                sqls.append(sql)

#                 sql = "UPDATE msm_BItem SET Description = MUID WHERE MUID = '" + boundary_id + "'"
#                 sqls.append(sql)

        for tracer in tracers:
            sqls.append("INSERT INTO msm_ADComponent (muid,altid,active,typeno,decayconst,unitno) SELECT '" + tracer + "', 0, 1, 1, 0, 2205")

        if not single_tracer:
            for i, row in tracer_ids.iterrows():
                muid = str(row["MUID"])
                new_zone = str(row["Tracer_Short"])
                sqls.append("UPDATE msm_Catchment SET Location = Location || '_" + new_zone + "' WHERE MUID = '" + muid + "'")
                sqls.append("UPDATE msm_Loadpoint SET LoadLocation = LoadLocation || '_" + new_zone + "', LoadSubCategory = LoadCategory || '_' || LoadLocation || '_" + new_zone + "' WHERE CatchmentID = '" + muid + "'")
        else:
            new_zone = tracer_z
            sqls.append("UPDATE msm_Catchment SET Location = Location || '_" + single_tracer_ids[1] + "'")
            sqls.append("UPDATE msm_Loadpoint SET LoadLocation = LoadLocation || '_" + single_tracer_ids[1] + "', LoadSubCategory = LoadCategory || '_' || LoadLocation || '_" + single_tracer_ids[1] + "'")

        execute_sql(sqls, mu_path)
        
        sql = "UPDATE msm_Project SET adinitcondtypeno = 1, decouplingadhdtypeno = 1 WHERE Enable_HD = 1"
        execute_sql(sql, mu_path)

        if runoff_tracer == True:
            sql = "UPDATE msm_Project SET Enable_AD = 1, Enable_Catchment = 0, Enable_RR = 0 WHERE Enable_HD = 1"
        else:
            sql = "UPDATE msm_Project SET Enable_AD = 1 WHERE Enable_HD = 1"
        execute_sql(sql, mu_path)
        
#         sql = "UPDATE msm_Project SET MUID = MUID || '" + sim_ext + "' WHERE Enable_HD = 1 AND LENGTH(MUID) < " + str(40 - len(sim_ext) + 1)
#         execute_sql(sql, mu_path)
        
        
        sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
        muids = list(sql_to_df(sql,mu_path).muid)
        sqls = []
        for muid in muids:
            if len(muid) <= 40 - len(sim_ext):
                muid_new = muid + sim_ext
                sqls.append("UPDATE msm_Project SET muid = '" + muid_new + "' WHERE muid = '" + muid + "'")        
                sqls.append("UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'") 
                sql = "INSERT INTO msm_ProjectOutput (muid, altid, active, simulationid, outputid, contentstypeno, "
                sql += "formatno, dtsave, dtsaveunitno, defaultsaveperiodno, savestartdate, saveenddate) "
                sql += "SELECT simulationid || '" + sim_ext + "', altid, active, simulationid, 'Default_Network_AD', 12, "
                sql += "1, 60, 1, 1, savestartdate, saveenddate "
                sql += "FROM msm_ProjectOutput WHERE simulationid = '" + muid_new + "' AND outputid = 'Default_Network_HD'"
                sqls.append(sql)
        execute_sql(sqls, mu_path)


In [ ]:
#PERMANENT CELL 9
MessageBox(None,'All cells ran successfully.', 'Done', 0)